In [1]:
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras 
import warnings

warnings.filterwarnings('ignore')
print(tf.__version__)
for model in sklearn, pd, keras, np ,mlp:
    print(model.__name__, model.__version__)

2.1.0
sklearn 0.20.2
pandas 0.24.2
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
numpy 1.17.4
matplotlib 2.1.2


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=2)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [3]:
# version 0.2 数据归一化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
x_train = std.fit_transform(x_train)
x_valid = std.transform(x_valid)
x_test = std.transform(x_test)

In [8]:
# tf.keras.models.Sequential()
# version dnn, 多层神经网络，循环添加层次。
class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super(CustomizedDenseLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        """构建所需要的参数"""
        self.kernel = self.add_weight(name="kernel", shape=(input_shape[1], self.units), initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias', shape=(self.units,), initializer='zeros', trainable=True)
        super(CustomizedDenseLayer, self).build(input_shape)

    def call(self, x):
        """完成正向计算"""
        return self.activation(x @ self.kernel + self.bias)
    def get_config(self):
        config = {"units": self.units, "activation": self.activation}
        base_config = super(CustomizedDenseLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
model = tf.keras.models.Sequential([
    CustomizedDenseLayer(30,activation='relu',input_shape=x_train.shape[1:]),
    CustomizedDenseLayer(1),
])

# sparse_categorical_crossentropy:  如果y是一个数，会把它转化成一个向量，如 y->index., y->one hot->[]
model.compile(loss='mean_squared_error',
              optimizer='adam')


In [9]:
# version 0.3 回调函数
# Tensorboard, Earlystopping, Modelcheckpoint
logdir = './reg_custome_layer_callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "housing_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3)
]

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
customized_dense_layer_2 (Cu (None, 30)                270       
_________________________________________________________________
customized_dense_layer_3 (Cu (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(x_train, y_train, epochs=100,validation_data=(x_valid, y_valid),callbacks=callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 80us/sample - loss: 2.0510 - val_loss: 0.6324
Epoch 2/100
11610/11610 [==============================] - 1s 73us/sample - loss: 0.5374 - val_loss: 0.4710
Epoch 3/100
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4492 - val_loss: 0.4333
Epoch 4/100
11610/11610 [==============================] - 0s 37us/sample - loss: 0.4274 - val_loss: 0.4177
Epoch 5/100
11610/11610 [==============================] - 0s 39us/sample - loss: 0.4119 - val_loss: 0.4096
Epoch 6/100
11610/11610 [==============================] - 0s 38us/sample - loss: 0.4049 - val_loss: 0.4022
Epoch 7/100
11610/11610 [==============================] - 0s 37us/sample - loss: 0.3977 - val_loss: 0.3991
Epoch 8/100
11610/11610 [==============================] - 0s 39us/sample - loss: 0.3929 - val_loss: 0.3958
Epoch 9/100
11610/11610 [==============================] - 0s 38us/sample - loss: 0.388

In [ ]:
def plot_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()
    
plot_learning_curve(history)

In [ ]:
model.evaluate(x_test,y_test)